In [18]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from chatsky_llm_autoconfig.algorithms.topic_graph_generation import CycleGraphGenerator
from chatsky_llm_autoconfig.graph import BaseGraph
from langchain.prompts import PromptTemplate
from typing import Optional, Dict, Any
import networkx as nx
from langchain_core.language_models.chat_models import BaseChatModel

load_dotenv() 

True

In [19]:
enhanced_graph_prompt = PromptTemplate.from_template(
    """
Create a dialogue graph for a {topic} conversation that will be used for training data generation. The graph must follow these requirements:

1. Dialogue Flow Requirements:
   - Each assistant message (node) must be a precise question or statement that expects a specific type of response
   - Each user message (edge) must logically and directly respond to the previous assistant message
   - All paths must maintain clear context and natural conversation flow
   - Avoid any ambiguous or overly generic responses

2. Graph Structure Requirements:
   - Must contain at least 2 distinct cycles (return paths)
   - Each cycle should allow users to:
     * Return to previous choices for modification
     * Restart specific parts of the conversation
     * Change their mind about earlier decisions
   - Include clear exit points from each major decision path
   
3. Core Path Types:
   - Main success path (completing the intended task)
   - Multiple modification paths (returning to change choices)
   - Early exit paths (user decides to stop)
   - Alternative success paths (achieving goal differently)

Example of a good cycle structure:
Assistant: "What size coffee would you like?"
User: "Medium please"
Assistant: "Would you like that hot or iced?"
User: "Actually, can I change my size?"
Assistant: "Of course! What size would you like instead?"

Format:
{{
    "edges": [
        {{
            "source": "node_id",
            "target": "node_id",
            "utterances": ["User response text"]
        }}
    ],
    "nodes": [
        {{
            "id": "node_id",
            "label": "semantic_label",
            "is_start": boolean,
            "utterances": ["Assistant message text"]
        }}
    ]
}}

Requirements for node IDs:
- Must be unique integers
- Start node should have ID 1
- IDs should increment sequentially

Return ONLY the valid JSON without any additional text or explanations.
"""
)

In [20]:
def generate_dialogue_graph(topic: str, gen_model: BaseChatModel, prompt: PromptTemplate) -> BaseGraph:
    """
    Генерирует граф диалога по заданной теме
    
    Args:
        topic: тема для генерации диалога
        gen_model: модель для генерации
    
    Returns:
        nx.DiGraph: сгенерированный граф
    """
    print(f"\n💫 Generating graph for topic: {topic}")
    
    try:
        graph_generator = CycleGraphGenerator(prompt=prompt)
        graph = graph_generator.invoke(topic=topic, model=gen_model)
        print(f"✓ Graph generated: {len(graph.graph_dict['nodes'])} nodes, {len(graph.graph_dict['edges'])} edges")
        
        return graph

    except Exception as e:
        print(f"❌ Generation error: {str(e)}")
        raise

In [21]:
def validate_graph_requirements(
    graph: BaseGraph,
    min_cycles: int = 2
) -> Dict[str, Any]:
    """
    Проверяет граф на соответствие техническим требованиям
    
    Args:
        graph: BaseGraph для проверки
        min_cycles: минимальное требуемое количество циклов
        
    Returns:
        Dict с результатами проверки:
        {
            "meets_requirements": bool,
            "cycles": List[List[int]],
            "cycles_count": int
        }
    """
    print("\n🔍 Checking graph requirements...")
    
    try:
        cycles = list(nx.simple_cycles(graph.graph))
        cycles_count = len(cycles)
        
        print(f"🔄 Found {cycles_count} cycles in the graph:")
        for i, cycle in enumerate(cycles, 1):
            print(f"Cycle {i}: {' -> '.join(map(str, cycle + [cycle[0]]))}")
            
        meets_requirements = cycles_count >= min_cycles
        
        if not meets_requirements:
            print(f"❌ Graph doesn't meet cycle requirements (minimum {min_cycles} cycles needed)")
        else:
            print("✅ Graph meets cycle requirements")
            
        return {
            "meets_requirements": meets_requirements,
            "cycles": cycles,
            "cycles_count": cycles_count
        }
        
    except Exception as e:
        print(f"❌ Validation error: {str(e)}")
        raise

In [22]:
from chatsky_llm_autoconfig.algorithms.dialogue_generation import DialogueSampler
from chatsky_llm_autoconfig.metrics.automatic_metrics import all_utterances_present

CYCLE_REQUIREMENT = 2

gen_model = ChatOpenAI(
        model='o1-mini',
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=1
    )
    
    # Генерация
graph = generate_dialogue_graph("ordering pizza", gen_model, enhanced_graph_prompt)
    
    # Проверка требований
validation_result = validate_graph_requirements(graph, min_cycles=CYCLE_REQUIREMENT)

dial_sampler = DialogueSampler()
sampled_dialogues = dial_sampler.invoke(graph, 1, -1)

res = all_utterances_present(graph, sampled_dialogues)

res


💫 Generating graph for topic: ordering pizza


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Graph generated: 8 nodes, 18 edges

🔍 Checking graph requirements...
🔄 Found 6 cycles in the graph:
Cycle 1: 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 1
Cycle 2: 1 -> 2 -> 3 -> 4 -> 6 -> 1
Cycle 3: 1 -> 2 -> 1
Cycle 4: 2 -> 3 -> 2
Cycle 5: 3 -> 4 -> 3
Cycle 6: 4 -> 5 -> 4
✅ Graph meets cycle requirements


False